In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

# 실제 크롤링할 URL (예시 URL, 실제 주소로 변경)
url = "https://downloadcenter.samsung.com/content/PM/202305/20230525172504861/KO/air-web_FAC029-02/content7.html#4605B2F8-DC3E-4EDC-BE73-4B2ABE233835"
response = requests.get(url)
response.encoding = "utf-8"  # 인코딩을 UTF-8로 설정
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

In [5]:
# 페이지 내에서 문제(증상)를 나타내는 h4 태그들을 찾습니다.
# (여기서는 예시로 h4 태그의 클래스 중 "Heading3-Noline_NoTOC_InContent"를 기준으로 합니다)
h4_tags = soup.find_all(
    "h4", class_=lambda c: c and "Heading3-Noline_NoTOC_InContent" in c
)

In [6]:
print(h4_tags)

[<h4 class="Heading3-Noline_NoTOC_InContent shap-target remove-space" data-class="Heading3 Noline NoTOC InContent" id="B3D0FAA1-DAD9-4EF2-BEEB-5A38D643D0C9">전원이 켜지지 않아요.</h4>, <h4 class="Heading3-Noline_NoTOC_InContent shap-target remove-space" data-class="Heading3 Noline NoTOC InContent" id="713BCD68-951F-4A93-9C67-2E953D1030D8">전원이 꺼지지 않아요.</h4>, <h4 class="Heading3-Noline_NoTOC_InContent shap-target remove-space" data-class="Heading3 Noline NoTOC InContent" id="3BE4929E-8EB4-4F78-8C30-7CCE1F18A451">온도 조절이 안 돼요.</h4>, <h4 class="Heading3-Noline_NoTOC_InContent shap-target remove-space" data-class="Heading3 Noline NoTOC InContent" id="052818BB-11A6-4C8A-B845-44A04BBFFD7A">바람 세기 조절이 안 돼요.</h4>, <h4 class="Heading3-Noline_NoTOC_InContent shap-target remove-space" data-class="Heading3 Noline NoTOC InContent" id="D96E2825-00A6-45D7-B296-6355BBCE2052">리모컨이 작동하지 않아요.</h4>, <h4 class="Heading3-Noline_NoTOC_InContent shap-target remove-space" data-class="Heading3 Noline NoTOC InContent" id="D

In [8]:
results = []
for h4 in h4_tags:
    # 증상은 h4 태그의 텍스트로 사용
    symptom = h4.get_text(strip=True)

    # 다음 형제 요소를 찾습니다.
    next_elem = h4.find_next_sibling()
    # 빈 텍스트 노드 건너뛰기
    while next_elem and (not getattr(next_elem, "name", None)):
        next_elem = next_elem.find_next_sibling()

    # Case 1: h4 바로 다음 요소가 Description 클래스라면 단일 항목으로 처리합니다.
    if next_elem and next_elem.get("class") and "Description" in next_elem.get("class"):
        cause = "Null"  # 원인 정보가 없으므로 Null 처리
        solution = next_elem.get_text(" ", strip=True)
        results.append({"증상": symptom, "원인": cause, "해결방안": solution})
    else:
        # Case 2: h4 이후부터 다음 h4 태그까지의 형제 요소들 중에서
        # UnorderList_1_Cell(원인)와 Description_Indent1(해결방안) 태그들이 쌍을 이룹니다.
        siblings = []
        for sib in h4.find_next_siblings():
            # 다음 h4 태그가 나오면 해당 영역은 종료합니다.
            if sib.name == "h4":
                break
            siblings.append(sib)

        # 해당 영역 내에서 두 종류의 태그를 각각 수집합니다.
        cause_cells = [
            elem
            for elem in siblings
            if elem.get("class") and "UnorderList_1_Cell" in elem.get("class")
        ]
        solution_divs = [
            elem
            for elem in siblings
            if elem.get("class") and "Description_Indent1" in elem.get("class")
        ]

        # 두 리스트의 순서대로 매칭합니다.
        for cell, sol_div in zip(cause_cells, solution_divs):
            # 원인 텍스트: cell 내부의 텍스트(불필요한 기호 등은 strip)
            cause_text = cell.get_text(" ", strip=True)
            # 해결방안 텍스트: 해당 div 내부의 텍스트
            solution_text = sol_div.get_text(" ", strip=True)
            results.append(
                {"증상": symptom, "원인": cause_text, "해결방안": solution_text}
            )

In [9]:
print(results)

[{'증상': '전원이 꺼지지 않아요.', '원인': 'Null', '해결방안': '실내기 맞춤건조 기능 실행 중에는 에어컨이 꺼지지 않습니다. 맞춤건조 기능이 끝날 때까지 잠시 기다려 주세요.'}, {'증상': '온도 조절이 안 돼요.', '원인': 'Null', '해결방안': '청정/스피드 운전 실행 중에는 온도를 조절할 수 없습니다. 버튼을 누르면 냉방 운전으로 변경됩니다.'}, {'증상': '바람 세기 조절이 안 돼요.', '원인': 'Null', '해결방안': '쾌적[AI(인공지능)쾌적]/제습/쾌면(열대야 쾌면) 운전이 실행되고 있거나 바람문이 1개만 운전하고 있을 때에는 바람 세기가 자동으로 설정됩니다. 단, 쾌적[AI(인공지능)쾌적] 운전 중 버튼을 누르면 냉방 운전으로 변경됩니다.'}, {'증상': '리모컨이 작동하지 않아요.', '원인': '• 실내기의 리모컨 수신부가 물건으로 가려져 있는 경우', '해결방안': '리모컨이 보내는 신호를 받을 수가 없습니다. 가린 물건을 치우고 실내기의 리모컨 수신부를 향해 리모컨을 눌러 주세요.'}, {'증상': '리모컨이 작동하지 않아요.', '원인': '• 건전지가 다 소모된 경우', '해결방안': '새 건전지로 교체해 주세요.'}, {'증상': '리모컨이 작동하지 않아요.', '원인': '• 강한 빛(삼파장 형광등, 네온사인등) 근처인 경우', '해결방안': '전파 방해로 리모컨이 작동되지 않을 수 있습니다. 형광등을 끄고 정상적으로 작동되는지 확인해 주세요.'}, {'증상': '리모컨 사용 시 벽걸이형 에어컨도 작동해요.', '원인': 'Null', '해결방안': '벽걸이형 에어컨이 스탠드형 에어컨과 가까운 곳에 있을 경우 리모컨 사용 시 벽걸이형 에어컨이 작동할 수 있습니다. 벽걸이형 에어컨이 영향을 받지 않도록 문을 닫거나 손으로 리모컨을 가려 사용해 주세요.'}, {'증상': '예약이 안 돼요.', '원인': 'Null', '해결방안': '예약 시간을 선택한 후에 반드시 버튼을 눌러야 예약 설정이